### Set autoreloading
This extension will automatically update with any changes to packages in real time

In [1]:
%load_ext autoreload
%autoreload 2

### Import packages
We'll need the `pynuml` and `nugraph` packages

In [2]:
import os
import sys
sys.path.append(f"{os.environ['HOME']}/nugraph/pynuml")
sys.path.append(f"{os.environ['HOME']}/nugraph/nugraph")
os.environ["NUGRAPH_DIR"] = f"{os.environ['HOME']}/nugraph"
os.environ["NUGRAPH_LOG"] = f"{os.environ['HOME']}/logs"
os.environ["NUGRAPH_DATA"] = f"{os.environ['HOME']}/data"
import pynuml
import nugraph as ng

### Open HDF5 event file

This file contains the low-level tables we need to process into graph objects

In [3]:
#file = pynuml.io.File("/exp/dune/data/users/hrazafin/iceberg/test_run9h5/NeutrinoML_test134.h5")
file = pynuml.io.File("/exp/dune/data/users/hrazafin/iceberg/run9_hdf5/run9_signal_extra_Sep2125/key_extra_run9_signal.h5")
#file = pynuml.io.File("/exp/dune/data/users/hrazafin/iceberg/run9_hdf5/h5_sg_bck/h5_merged_sg_gb.h5")

In [4]:

import h5py
import numpy as np

#file_path = "/exp/dune/data/users/hrazafin/iceberg/run9_hdf5/run9_signal_extra_Sep2125/key_extra_run9_signal.h5"
#307673 ; events = 463758

#file_path = "/exp/dune/data/users/hrazafin/iceberg/run9_hdf5/run9_signal_extra_Sep2125/full_signal_run9.h5"
#hit_table : 385134 ; events : 582998

file_path = "/exp/dune/data/users/hrazafin/iceberg/run9_hdf5/run9_signal_extra_Sep2125/run9_old_signal_iceberg.h5"

with h5py.File(file_path, "r") as f:
    hit_table = f["hit_table"]
    # Load only the 'event_id' field (may be named differently — check with list(hit_table.dtype.names))
    event_ids = hit_table["event_id"]
    unique_event_ids = np.unique(event_ids)
    print(f"Total unique event IDs in hit_table: {len(unique_event_ids)}")

    event_ids = f['event_table/event_id']
    print("Total events:", len(event_ids))

    print("Example event IDs:", event_ids[:10])

Total unique event IDs in hit_table: 78947
Total events: 119240
Example event IDs: [[      3       1 1604011]
 [      3       1 1604061]
 [      3       1 1604083]
 [      3       1 1604097]
 [      3       1 1604098]
 [      3       1 1604119]
 [      3       1 1604122]
 [      3       1 1604139]
 [      3       1 1604320]
 [      3       1 1604333]]


### Create hit graph producer

Create a class that can read input information and use it to generate graphs

In [5]:
semantic_labeller = pynuml.labels.StandardLabels()
#event_labeller = pynuml.labels.FlavorLabels()
event_labeller = pynuml.labels.StoppingMuon()
#signal_bkg = pynuml.labels.StoppingMuon()
processor = pynuml.process.HitGraphProducer(
        file=file,
        semantic_labeller=semantic_labeller,
        event_labeller = event_labeller, # default None
        label_vertex=False,
        label_position=True, # True is to activate 3D, default false
        planes=['u','v','y'],
        node_feats = ['integral','rms','tpc'],
        lower_bound = 20,
        store_detailed_truth = False
        #signal_bkg = signal_bkg
)

## Configure plotting utility
Instantiate the **pynuml** utility for plotting graph objects, which will allow us to visualise the graphs we create

In [6]:
plot = pynuml.plot.GraphPlot(
    planes=('u', 'v', 'y'),
    classes=semantic_labeller.labels[:-1])

### Load a buffer of input graphs

For interactive testing, we can use `read_data` to load the first 100 events in the event file using the following syntax, which will return a list of events that we can use to initialise an iterator

In [7]:
file.read_data(0, 100)
#file.read_data_all()
evts = iter(file.build_evt())

In [8]:
levts = list(file.build_evt())
print("Events yielded by build_evt():", len(levts))

Events yielded by build_evt(): 64


### Retrieve the next event

This block retrieves an event from the buffer. Since we defined `evts` as an iterator over our 100 events, the following block can be executed multiple times, and each time it's executed, it will step to the next event in the buffer. After fetching an event, we pass it into the processor to create a graph. Not all events produce good graphs, so if the next event does not return a valid graph, the code will continue to fetch events until it finds the next event that produces a valid graph.

In [16]:
name, data = processor(next(evts))
skip = 0
while data is None:
    name, data = processor(next(evts))
    skip += 1
if skip > 0:
    print(f'skipped {skip} events that did not produce valid graphs.')

STOPPING MUON filter
✅ ***** Signal ***** ✅
primary -13
primary.end_position_x =  16.128517150878906
primary.end_position_y =  10.40115737915039
primary.end_position_z =  78.85472106933594
skipped 1 events that did not produce valid graphs.


### Visualise graph

We can use the plotting utility to take a closer look at the graph we just produced.

In [ ]:
fig = plot.plot(data, target='instance', how='true', filter='show')
fig

In [15]:
fig = plot.plot(data, target='semantic', how='true')
fig

TypeError: 'NoneType' object is not subscriptable

In [12]:
#fig = plot.plot(data, target='instance', how='true', xyz='true')
##### 3D views
fig = plot.plot(data, target='instance', how='true',  filter='show',xyz='true')
fig

FigureWidget({
    'data': [{'customdata': array([['muon', '1', 30.960250854492188, -278.4313049316406],
                                   ['muon', '1', 31.439250946044922, -277.3503723144531],
                                   ['muon', '1', 31.918249130249023, -275.9581604003906],
                                   ...,
                                   ['muon', '1', -11.624770164489746, -278.3617858886719],
                                   ['muon', '1', -10.691370010375977, -279.03497314453125],
                                   ['muon', '1', -10.22467041015625, -279.1280822753906]], dtype=object),
              'hovertemplate': ('instance truth=%{customdata[1]' ... 'tomdata[3]:.1f}<extra></extra>'),
              'legendgroup': '1, muon',
              'marker': {'color': '#FF97FF', 'size': 1, 'symbol': 'circle'},
              'mode': 'markers',
              'name': '1, muon',
              'scene': 'scene',
              'showlegend': True,
              'type': 'scatter3d'

In [13]:
file.read_data(1, 80)
nevts = iter(file.build_evt())
i=0
while i < 50:
     name, data = processor(next(nevts))
     i +=1
     #print('event number =', i)
     print("_____________________________________________")
name, data = processor(1)

_____________________________________________
STOPPING MUON filter
❌ ***** Background ***** ❌
primary -13
_____________________________________________
_____________________________________________
STOPPING MUON filter
✅ ***** Signal ***** ✅
primary -13
primary.end_position_x =  16.128517150878906
primary.end_position_y =  10.40115737915039
primary.end_position_z =  78.85472106933594
_____________________________________________
_____________________________________________
STOPPING MUON filter
❌ ***** Background ***** ❌
primary -13
_____________________________________________
STOPPING MUON filter
✅ ***** Signal ***** ✅
primary -13
primary.end_position_x =  13.777453422546387
primary.end_position_y =  34.484169006347656
primary.end_position_z =  83.3642807006836
_____________________________________________
STOPPING MUON filter
✅ ***** Signal ***** ✅
primary 13
primary.end_position_x =  15.488927841186523
primary.end_position_y =  0.9221203327178955
primary.end_position_z =  33.390850

TypeError: 'int' object is not subscriptable